In [1]:
import pandas as pd
import numpy as np
import requests
import re
from numba import njit
import pickle
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [2]:
class CreateDistance(object):
    def __init__(self):
        try:
            read_city = open("city.pkl","rb")
            self.city = pickle.load(read_city)
        except:
            self.city = self.load_city()
        
        try:
            read_distance = open("distance.pkl","rb")
            self.distance = pickle.load(read_distance)            
        except:
            self.distance = self.create_distance_mat()      
       
        
    def load_city(self):
        #读取城市信息
        city = pd.read_excel("city.xlsx")
        city = city["city,longitude,latitude"].str.split(",",expand = True)
        city.columns = ["city","longitude","latitude"]
        output = open("city.pkl","wb")
        pickle.dump(city,output)
        return city
        
    #@njit(fastmath = True)
    def compute_distance(self,longitude1,latitude1,longitude2,latitude2):
        header = {'user_agent':'Mozilla/5.0 (Windows NT 10.0;WOW64)\
                       AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}
        #高德地图API
        request_url = 'http://restapi.amap.com/v3/distance?key=6c4e7caf822a3add82fdd0a71c4258e9&origins='\
        +str(longitude1) + ',' + str(latitude1) + '&destination=' + str(longitude2) + ',' + str(latitude2) +'&type=1'
        data = requests.get(request_url,headers = header,timeout = 100)
        data.encoding = 'utf-8'
        data = data.text
        #"distance":"2646","duration":"660"
        #匹配返回值中的距离和时间
        pattern = '"distance":"(.*?)","duration":"(.*?)"'
        result = re.findall(pattern,data)
        #返回距离result[0][0],返回时间result[0][1]
        return result[0][0]
    
    #@njit(fastmath = True)
    def create_distance_mat(self):
        #加载城市&经纬度
        data = self.city         
        #记录间距离的矩阵
        distance_mat = np.zeros(shape =(data.shape[0],data.shape[0]),dtype = np.float64)
        #已计算距离城市记录
        computed = {}
        for i in range(data.shape[0]):            
            for j in range(data.shape[0]):
                if i == j:
                    continue
                if (i,j) in computed:
                    continue
                else:
                    computed[(i,j)] = self.compute_distance(data.iloc[i]["longitude"],data.iloc[i]["latitude"],
                                                           data.iloc[j]["longitude"],data.iloc[j]["latitude"])
                     
                    computed[(j,i)] = computed[(i,j)]
                    distance_mat[i,j] = computed[i,j]
                    distance_mat[j,i] = computed[i,j]
        #存储距离矩阵
        output = open("distance.pkl","wb")
        pickle.dump(distance_mat,output)
        return distance_mat

In [3]:
class TSP(CreateDistance):
    def __init__(self):
        super().__init__()        
        self.city_names = self.city["city"].values.tolist()        
    
    #设置数据
    def create_data_model(self):
        data = {}
        data["distance_matrix"] = self.distance / 1000
        data["number_vehicles"] = 4
        data["depot"] = 3
        return data
    
    #得到解决方案
    def get_solution(self,manager,routing,solution,number_vehicles):
        distance = dict() 
        plan = dict()
        for vehicle_id in range(number_vehicles):
            index = routing.Start(vehicle_id)
            distance[vehicle_id] = 0
            plan[vehicle_id] = []
            while not routing.IsEnd(index):
                node = manager.IndexToNode(index)
                plan[vehicle_id].append(self.city_names[node])
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                distance[vehicle_id] += routing.GetArcCostForVehicle(previous_index,index,vehicle_id)
            plan[vehicle_id].append(self.city_names[manager.IndexToNode(index)])
        #求总行车距离
        total_distance = 0
        for d in distance:
            total_distance += distance[d]
        #求总行经城市
        total_city = 1 #出发城市
        for p in plan:
            total_city += len(plan[p])-2 #每个计划减掉出发城市
        return plan,distance,total_city,total_distance
    
    def tsp(self):
        #初始化数据
        data = self.create_data_model()
        
        #创建路径管理：tsp_size(城市数量)，num_vehicles(车辆数量)，depot(原点)
        manager = pywrapcp.RoutingIndexManager(len(data["distance_matrix"]),
                                              data["number_vehicles"],data["depot"])
        #创建routing_model
        routing = pywrapcp.RoutingModel(manager)
        
        #计算两点之间的距离
        def distance_callback(from_index,to_index):
            #Convert from routing variable Index to distance matrix NodeIndex
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data["distance_matrix"][from_node][to_node]
        
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        
        #定义损失
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
        
        #添加距离约束
        dimension_name = "Distance"
        #每辆车最大行驶距离10000公里
        routing.AddDimension(transit_callback_index,
                            0,
                            10000,
                            True,
                            dimension_name)
        
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        #尽量减少车辆间的最大距离
        distance_dimension.SetGlobalSpanCostCoefficient(100)
        
        #模型参数设置
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
        
        #求解路径规划
        solution = routing.SolveWithParameters(search_parameters)   
        
        #获得求解结果
        if solution:            
            return self.get_solution(manager,routing,solution,data["number_vehicles"])
        else:
            return "求解失败"       
        

In [4]:
tsp = TSP()
plan,routing_distance,total_city,total_distance =tsp.tsp()
#打印行车路线
for p in plan:
    routing =""
    for i in range(len(plan[p])-1):
        routing += "{}->".format(plan[p][i])
    routing += plan[p][-1]
    print("第{}辆车的行车路线：".format(p + 1))
    print(routing)
    
#总遍历城市
print("4辆车经过的城市总数：",total_city)

#每辆车行车距离
for d in routing_distance:
    print("第{}辆车的行车距离为：{}公里".format(d,routing_distance[d]))
    
#四辆车总的行车距离
print("4辆车总的行车距离：{}公里".format(total_distance))

第1辆车的行车路线：
北京市->太原市->银川市->乌鲁木齐市->呼和浩特市->北京市
第2辆车的行车路线：
北京市->石家庄市->西安市->兰州市->西宁市->拉萨市->北京市
第3辆车的行车路线：
北京市->成都市->昆明市->贵阳市->重庆市->长春市->哈尔滨市->沈阳市->北京市
第4辆车的行车路线：
北京市->天津市->济南市->合肥市->南京市->上海市->杭州市->福州市->香港->澳门->广州市->海口市->南宁市->长沙市->南昌市->武汉市->郑州市->北京市
4辆车经过的城市总数： 33
第0辆车的行车距离为：5994公里
第1辆车的行车距离为：7479公里
第2辆车的行车距离为：7724公里
第3辆车的行车距离为：7321公里
4辆车总的行车距离：28518公里
